In [9]:
import json
import re
import random
import itertools
from string import Formatter

In [3]:
with open('./ner_dataset/fallback.json') as f:
    data = json.load(f)

    intents = list(data.keys())

In [4]:
intents

['Variable Declaration',
 'Constant Declaration',
 'Function Declaration',
 'Class Declaration',
 'Assignment Operation',
 'Conditional Operation',
 'For Loop',
 'While Loop',
 'Bitwise Operation',
 'Mathematical Operation',
 'Membership Operation',
 'Casting',
 'Input',
 'Output',
 'Assertion',
 'Libraries',
 'File System',
 'IDE Operation',
 'Comment',
 'Interactive Commands',
 'Git Operation',
 'Activate Mouse',
 'Activate Interactive',
 'Exit Block',
 'Mouse Click',
 'Array Operation']

In [11]:
templates = {
    "book_flight": r"Book a flight to (?P<destination>[\w\s]+)",
    "get_weather": r"What's the weather in (?P<location>[\w\s]+)",
    "play_music": r"Play (?P<song>.+)"
}

def classify_intent_and_extract_entities(text):
    for intent, pattern in templates.items():
        match = re.match(pattern, text)
        if match:
            entities = match.groupdict()
            return intent, entities
    return "unknown", {}

# Example usage
text = "Book a flight to New York"
intent, entities = classify_intent_and_extract_entities(text)
print(f"Intent: {intent}")
print(f"Entities: {entities}")


Intent: book_flight
Entities: {'destination': 'New York'}


In [12]:
import re

# Define the templates with placeholders for slots
templates = {
    "book_flight": [
        r"Book a flight from (?P<origin>[\w\s]+) to (?P<destination>[\w\s]+)",
        r"Book a flight to (?P<destination>[\w\s]+)"
    ],
    "get_weather": [
        r"What's the weather in (?P<location>[\w\s]+)",
        r"How is the weather in (?P<location>[\w\s]+)"
    ],
    "play_music": [
        r"Play (?P<song>.+) by (?P<artist>.+)",
        r"Play the song (?P<song>.+)"
    ]
}

def classify_intent_and_extract_entities(text):
    for intent, patterns in templates.items():
        for pattern in patterns:
            match = re.match(pattern, text)
            if match:
                entities = match.groupdict()
                return intent, entities
    return "unknown", {}

# Example usage
texts = [
    "Book a flight from San Francisco to New York",
    "Book a flight to New York",
    "What's the weather in London",
    "How is the weather in Tokyo",
    "Play Shape of You by Ed Sheeran",
    "Play the song Shape of You"
]

for text in texts:
    intent, entities = classify_intent_and_extract_entities(text)
    print(f"Text: {text}")
    print(f"Intent: {intent}")
    print(f"Entities: {entities}")
    print()


Text: Book a flight from San Francisco to New York
Intent: book_flight
Entities: {'origin': 'San Francisco', 'destination': 'New York'}

Text: Book a flight to New York
Intent: book_flight
Entities: {'destination': 'New York'}

Text: What's the weather in London
Intent: get_weather
Entities: {'location': 'London'}

Text: How is the weather in Tokyo
Intent: get_weather
Entities: {'location': 'Tokyo'}

Text: Play Shape of You by Ed Sheeran
Intent: play_music
Entities: {'song': 'Shape of You', 'artist': 'Ed Sheeran'}

Text: Play the song Shape of You
Intent: play_music
Entities: {'song': 'Shape of You'}



In [14]:
import re

pattern = r"(?P<first_name>\w+) (?P<last_name>\w+)"
text = "John Doe"

match = re.match(pattern, text)
if match:
    first_name = match.group('first_name')
    last_name = match.group('last_name')
    print(f"First Name: {first_name}")
    print(f"Last Name: {last_name}")


First Name: John
Last Name: L


In [16]:
intents_dict = {
    "variable" : [ 
    "{declare} a {new} variable {named} {name} and {assign} {value}",
    "{declare} a {new} {type} variable {named} {name} and {assign} {value}",
    "{declare} a {new} {type} variable {named} {name} and {assign} {value}"
  ]}

template = intents_dict['variable'][0]

filtered_parameters = {field_name for _, field_name, _, _ in Formatter().parse(template) if field_name}

filtered_parameters

{'assign', 'declare', 'name', 'named', 'new', 'value'}

Done:
- variable
- constant
- class
- assignment
- for
- bitwise
- membership
- libraries
- casting
- output
- input
- assertion (check condition)
- git
- ide
- interactive commands
- mouse click
- comment
- activate mouse
- activate interactive
- file
- while
- function
- conditional
- mathematical


In [39]:
import re
import json

# # Load templates from JSON file
templates = {
"libraries": [
    "(declare|create|define|make) (a[n]? )?(new )?constant (named|called|with name) (?P<name>[\\w\\s]+?) and (assign|set|value) (it to )?(?P<value>.+)",
    "(declare|create|define|make) (a[n]? )?(new )?constant (named|called|with name) (?P<name>[\\w\\s]+?) and (assign|set|value) (it to )?(?P<value>.+)",
    "(declare|create|define|make) (a[n]? )?(new )?constant (?P<name>[\\w\\s]+?) (and|with) (assign|set|value) (it to )?(?P<value>.+)",
    "(declare|create|define|make) (a[n]? )?(new )?(?P<type>\\w+) constant (named|called|with name) (?P<name>[\\w\\s]+?) and (assign|set|value) (it to )?(?P<value>.+)",
    "(declare|create|define|make) (a[n]? )?(new )?(?P<type>\\w+) constant  (?P<name>[\\w\\s]+?) and (assign|set|value) (it to )?(?P<value>.+)",
    "(declare|create|define|make) (a[n]? )?(new )?(?P<type>\\w+) constant (?P<name>[\\w\\s]+?) and (assign|set|value) (it to )?(?P<value>.+)",
    "(declare|create|define|make) (a[n]? )?(new )?(?P<type>\\w+) constant (named|called|with name) (?P<name>[\\w\\s]+)",
    "(declare|create|define|make) (a[n]? )?(new )?constant (?P<name>[\\w\\s]+) and (assign|set|value) (it to )?(?P<value>.+) (and|with) (type|datatype) (?P<type>\\w+)",
    "(declare|create|define|make) (a[n]? )?(new )?constant (?P<name>[\\w\\s]+) and (assign|set|value) (it to )?(?P<value>.+)",
    "(declare|create|define|make) (a[n]? )?(new )?constant (?P<name>[\\w\\s]+) (with type|as) (?P<type>\\w+)"
    ]
}

with open('./ner_dataset/fallback_ner.json') as f:
    intent_templates = json.load(f)

    intent = 'comment'

    templates = intent_templates[intent]

    print(templates)



def extract_entities_from_regex(text, intent, patterns):
    for i, pattern in enumerate(patterns):
        print(i)
        match = re.match(pattern, text, re.IGNORECASE)  # Case-insensitive matching
        if match:
            entities = match.groupdict()
            return intent, entities
    return intent, {}


# Example usage
texts = [
    'add the comment this code computes the fibonacci sequence',
]

fallback_commands = [

"add the comment this code computes the fibonacci sequence"
]


for text in fallback_commands:
    intent, entities = extract_entities_from_regex(text, intent, templates)
    print(f"Text: {text}")
    print(f"Intent: {intent}")
    print(f"Entities: {entities}")
    print()


['(add |write |comment )?(the )?(comment )?(?P<comment>.+)']
0
Text: add the comment this code computes the fibonacci sequence
Intent: comment
Entities: {'comment': 'this code computes the fibonacci sequence'}



In [93]:
# extract entities given intent and template if the pattern matches
fallback_commands ={}

if fallback_commands:
    print('yes')
else:
    print('no')

no
